# Preparando dado texto com sklearn

Dados de texto requerem preparação especial antes de poder começar a usá-los para modelagem preditiva. O texto deve ser analisado para remover palavras, chamado de tokenização. Então as palavras precisam ser codificadas como números inteiros (`int`) ou valores de ponto flutuante (`float`) para uso como entrada em um algoritmo de aprendizado de máquina, chamado extração de recursos (ou vetorização).


A biblioteca **scikit-learn** oferece ferramentas fáceis de usar para realizar tanto tokenização quanto extração de recursos dos seus dados de texto. Aqui vamos estudar um pouco sobre elas, descobrindo exatamente como pode-se preparar os dados de texto para modelagem preditiva em Python com scikit-learn. A ideia aqui é abordar:

* Como converter texto para vetores de contagem de palavras com `CountVectorizer`.
* Como converter texto para vetores de frequência de palavras com `TfidfVectorizer`.
* Como converter texto para inteiros únicos com `HashingVectorizer`.

## Documentação:
* https://scikit-learn.org/stable/index.html

In [38]:
import pandas as pd

In [64]:
def summary(vetor):
    '''
    Função criada para facilitar o processo
    de sumarização dos elementos tratados nesse
    estudo.

    # Entrada:
    Objeto.scipy: Vetor transformado.
    '''
    try:
        print(vetor.shape)
    except Exception as e:
        print(f"Ocorreu um erro: {type(e).__name__} - {e}")
    
    try:
        print(type(vetor))
    except Exception as e:
        print(f"Ocorreu um erro: {type(e).__name__} - {e}")
    
    try:
        print(vetor.toarray())
    except Exception as e:
        print(f"Ocorreu um erro: {type(e).__name__} - {e}")

# Contagem de palavras com `CountVectorizer`:

O `CountVectorizer` fornece uma maneira simples de tokenizar uma coleção de documentos de texto e construir um vocabulário de palavras conhecidas, mas também de codificar (*encode*) novos documentos usando esse vocabulário. Um vetor codificado (*encode*) é retornado com um comprimento do vocabulário inteiro e uma contagem inteira para o número de vezes que cada palavra apareceu no documento.

Porque essas vetores (Matrizes) conterão muitos zeros, chamamos eles de **esparsos**. Esta implementação produz uma representação esparsa das contagens usando `scipy.sparse.csr_matrix` (Se você não fornecer um dicionário a priori e não usar um analisador que faça algum tipo de seleção de recursos, **o número de recursos será igual ao tamanho do vocabulário encontrado na análise dos dados**). Os vetores retornados de uma chamada para `transform()` serão vetores esparsos, e você pode transformá-los de volta para arrays NumPy para olhar e entender melhor o que está acontecendo chamando a função `toarray()`.

**CountVectorizer:**

* **Fit:** Durante a etapa de fit, ele aprende o vocabulário, ou seja, todas as palavras únicas presentes no conjunto de treinamento.
* **Transform:** Depois, ele usa esse vocabulário para transformar novos documentos em vetores de contagem de termos.

In [1]:
# Importando a classe CountVectorize:
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Frase:
text = ["The quick brown fox jumped over the lazy dog."]

In [5]:
# tokenize and build vocab
vectorizer.fit(text)

CountVectorizer()

In [6]:
# summarize
print(vectorizer.vocabulary_)

{'the': 7, 'quick': 6, 'brown': 0, 'fox': 2, 'jumped': 3, 'over': 5, 'lazy': 4, 'dog': 1}


In [10]:
# encode document
vector = vectorizer.transform(text)

In [63]:
# summarize encoded vector
summary(vector)

(3, 20)
<class 'scipy.sparse._csr.csr_matrix'>
[[ 0.          0.          0.          0.          0.          0.33333333
   0.         -0.33333333  0.33333333  0.          0.          0.33333333
   0.          0.          0.         -0.33333333  0.          0.
  -0.66666667  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.         -0.70710678  0.          0.          0.          0.
  -0.70710678  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.         -0.70710678  0.          0.
  -0.70710678  0.        ]]


Importantemente, o mesmo vetorizador pode ser usado em documentos que contenham palavras não incluídas no vocabulário. Essas palavras são ignoradas e nenhuma contagem é dada no vetor resultante. Por exemplo, abaixo está um exemplo de uso do vetorizador acima para codificar um documento com uma palavra no vocabulário e uma palavra que não é.

In [11]:
# encode another document
text2 = ["the puppy"]
vector = vectorizer.transform(text2)
print(vector.toarray())

[[0 0 0 0 0 0 0 1]]


# Frequências de palavras com TfidfVectorizer

Contagens de palavras são um bom ponto de partida, mas são muito básicas. Um problema com contagens simples é que algumas palavras como "*the*" aparecerão muitas vezes e suas grandes contagens não serão muito significativas nos vetores codificados. Uma alternativa é calcular frequências de palavras, e o método mais popular é chamado de **TF-IDF**. Isso é um acrônimo que significa ***Term Frequency*** - ***Inverse Document Frequency***, que são os componentes das pontuações resultantes atribuídas a cada palavra.

* **Term Frequency:** Isso resume quantas vezes uma determinada palavra, ou termo, $t$ aparece dentro de um documento $d$. É uma razão simples, mesma lógica da ideia da **probabilidade de Laplace**, ou *Naive Definition of Probability*.
$$\text{tf}{(t,d)} = \frac{\text{Numero de vezes que o termo}\ t\ \text{aparece no texto}\ d}{\text{Numero total de termos no texto}\ d}$$

* **Inverse Document Frequency:** Isso reduz a escala de palavras que aparecem muito em documentos.
$$\text{idf}(t) = \log\left(\frac{1+n}{1+df(t)}\right) +1$$

Onde,
* $n$ é o número total de documentos/textos;
* $df(t)$ é o número de documentos que contêm o termo $t$.

De forma simples, **TF-IDF** são pontuações de frequência de palavras que tentam destacar palavras que são mais interessantes, por exemplo, frequentes em um documento, mas não em documentos, se dá, de forma completa, como, 

$$tf\text{-}idf(t,d) = tf(t,d) \times idf(t)$$

**TfidfVectorizer:**

* **Fit:** Similar ao CountVectorizer, ele aprende o vocabulário e calcula as estatísticas necessárias para o IDF (frequência inversa de documentos).
* **Transform:** Usa o vocabulário e as estatísticas aprendidas para transformar novos documentos em vetores TF-IDF.

In [12]:
# Importando classe:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
# lista de frases
text = ["The quick brown fox jumped over the lazy dog.", "The dog.", "The fox"]

In [15]:
# instanciando transformador:
vectorizer = TfidfVectorizer()

# tokenizando e construindo vocabulário:
vectorizer.fit(text)

TfidfVectorizer()

In [37]:
# Sumário
df = pd.DataFrame(vectorizer.vocabulary_.values(), columns=["words"])
df = pd.concat([df,pd.DataFrame(vectorizer.idf_, columns=["idf"])], axis=1)
df.index = vectorizer.vocabulary_.keys();df

,words,idf
the,7,1.693147
quick,6,1.287682
brown,0,1.287682
fox,2,1.693147
jumped,3,1.693147
over,5,1.693147
lazy,4,1.693147
dog,1,1.000000


In [66]:
# Codificando documneto:
print(text[0])
vector = vectorizer.transform([text[0]])

The quick brown fox jumped over the lazy dog.


In [67]:
# Sumário do vetor codificado:
summary(vector)

(1, 20)
<class 'scipy.sparse._csr.csr_matrix'>
[[ 0.          0.          0.          0.          0.          0.33333333
   0.         -0.33333333  0.33333333  0.          0.          0.33333333
   0.          0.          0.         -0.33333333  0.          0.
  -0.66666667  0.        ]]


# Transforme uma matriz de contagem em uma representação normalizada de tf ou tf-idf com `TfidfTransformer`

Bascimente a ideia do `TfidfTransformer` no scikit-learn é usado para transformar uma matriz de contagem de termos (como a produzida pelo `CountVectorizer`) em uma matriz **TF-IDF**.

**OBS:**  Esse método não tem nada a ver com a arquitetura transformers, é só uma conhecidencia os nomes. Inclusive é importante ressaltar que esses métodos são métodos deterministicos. **Não tem nada de redes neurais aqui!**

**TfidfTransformer:**

* **Fit:** Aprende as estatísticas de IDF a partir da matriz de contagem de termos fornecida.
* **Transform:** Transforma a matriz de contagem de termos em uma matriz TF-IDF usando as estatísticas aprendidas.

In [50]:
# Importando TfidfTransformer:
from sklearn.feature_extraction.text import TfidfTransformer

In [47]:
# Instanciando CountVectorizer:
count_vect = CountVectorizer()

# Gerando Contagem:
X_counts = count_vect.fit_transform(text)

# Sumário
print(vectorizer.vocabulary_)

{'the': 7, 'quick': 6, 'brown': 0, 'fox': 2, 'jumped': 3, 'over': 5, 'lazy': 4, 'dog': 1}


In [71]:
# Instanciando TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Gerando normalização:
X_tfidf = tfidf_transformer.fit_transform(X_counts)

In [73]:
# Sumário do vetor codificado:
summary(X_tfidf)

(3, 8)
<class 'scipy.sparse._csr.csr_matrix'>
[[0.36388646 0.27674503 0.27674503 0.36388646 0.36388646 0.36388646
  0.36388646 0.42983441]
 [0.         0.78980693 0.         0.         0.         0.
  0.         0.61335554]
 [0.         0.         0.78980693 0.         0.         0.
  0.         0.61335554]]


Note que ele gera que o `TfidfTransformer` gera uma matriz esparça. Acho que dá pra simplificar como o mesmo resultado do método `TfidfVectorizer`, porém, menos trabalhado.

#  Hashing com HashingVectorizer

Métodos baseados tokenização de palavras, como sabemos, com frequência geral matrizes esparças (e hoje em dia nem são tão usados). Uma solução inteligente para lidar contornar possíveis vetores de tamanhos que desacelerem os algorítmos é usar um **hash unidirecional** de palavras para convertê-las em inteiros. 

* **Vantagem:** A **parte inteligente** é que **não é necessário vocabulário e você pode escolher um vetor de comprimento fixo arbitrário**. 

* **Desvantagem:** Uma desvantagem é que **o hash é uma função unidirecional**, então **não há como converter a codificação de volta para uma palavra** (o que pode não importar para muitas tarefas de aprendizado supervisionado).

A classe `HashingVectorizer` implementa essa abordagem que pode ser usada para fazer hash de palavras de forma consistente, depois tokenizar e codificar documentos conforme necessário.

In [74]:
# Importando classe HashingVectorizer:
from sklearn.feature_extraction.text import HashingVectorizer

In [75]:
# Instanciando Classe:
vectorizer = HashingVectorizer(n_features=20)

In [76]:
# Codificando documento:
vector = vectorizer.transform(text)

In [77]:
# Sumário do vetor codificado:
summary(vector)

(3, 20)
<class 'scipy.sparse._csr.csr_matrix'>
[[ 0.          0.          0.          0.          0.          0.33333333
   0.         -0.33333333  0.33333333  0.          0.          0.33333333
   0.          0.          0.         -0.33333333  0.          0.
  -0.66666667  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.         -0.70710678  0.          0.          0.          0.
  -0.70710678  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.         -0.70710678  0.          0.
  -0.70710678  0.        ]]
